In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE234017"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE234017"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE234017.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE234017.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE234017.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Spatial Transcriptomics Suggests That Alterations 4 Occur in the Preneoplastic Breast Microenvironment of 5 Q2 BRCA1/2 Mutation Carriers"
!Series_summary	"Breast cancer is the most common cancer in females, affecting one in every eight women and accounting for the majority of cancer-related deaths in women worldwide. Germline mutations in the BRCA1 and BRCA2 genes are significant risk factors for specific subtypes of breast cancer. BRCA1 mutations are associated with basal-like breast cancers, whereas BRCA2 mutations are associated with luminal-like disease. Defects in mammary epithelial cell differentiation have been previously recognized in germline BRCA1/2 mutation carriers even before cancer incidence. However, the underlying mechanism is largely unknown. Here, we employ spatial transcriptomics to investigate defects in mammary epithelial cell differentiation accompanied by distinct microenvironmental alterations in preneoplastic breast tissue

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the context, this is a spatial transcriptomics study examining BRCA1/BRCA2 carriers,
# which includes gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 & 2.2 Data Availability and Type Conversion

# For trait (Breast Cancer - considering BRCA mutation status)
# We can use row 2 which contains 'genotype: WT', 'genotype: BRCA1', 'genotype: BRCA2'
trait_row = 2

def convert_trait(value):
    """
    Convert genotype values to binary: 
    - WT (wild type/control) = 0
    - BRCA1/BRCA2 mutation = 1
    """
    if isinstance(value, str) and ':' in value:
        genotype = value.split(':', 1)[1].strip()
        if 'WT' in genotype:
            return 0
        elif 'BRCA1' in genotype or 'BRCA2' in genotype:
            return 1
    return None

# For age - Age information is not available in the sample characteristics
age_row = None

def convert_age(value):
    """Placeholder function since age data is not available"""
    return None

# For gender - Gender information is not explicitly provided, but this is breast tissue,
# so we can infer it's from female patients
# However, since it would be a constant value across all samples, 
# we'll consider it as not available for analytical purposes
gender_row = None

def convert_gender(value):
    """Placeholder function since gender data is not available"""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary contains keys as row indices and values as lists of characteristics
    sample_chars_dict = {
        0: ['patient: WT.4', 'patient: BRCA2.4', 'patient: BRCA1.4', 'patient: BRCA2.1', 'patient: WT.1', 'patient: BRCA1.1', 'patient: BRCA2.2', 'patient: BRCA1.2', 'patient: WT.2', 'patient: WT.3', 'patient: BRCA2.3', 'patient: BRCA1.3'],
        1: ['tissue segment: epithelium', 'tissue segment: stroma'],
        2: ['genotype: WT', 'genotype: BRCA2', 'genotype: BRCA1'],
        3: ['scan batch: S1', 'scan batch: S5', 'scan batch: S6', 'scan batch: S9', 'scan batch: S13']
    }
    
    # Create a DataFrame with appropriate samples as columns
    # We'll use the first row (patient IDs) to create sample names
    samples = []
    for patient in sample_chars_dict[0]:
        patient_id = patient.split(': ')[1].strip()
        samples.append(patient_id)
    
    # Create a DataFrame with characteristics as rows and samples as columns
    clinical_data = pd.DataFrame(index=range(len(sample_chars_dict)), columns=samples)
    
    # Fill in the DataFrame with characteristic values
    for row_idx, chars in sample_chars_dict.items():
        for char in chars:
            if ': ' in char:
                value, label = char.split(': ', 1)
                # For each sample, if its name contains the label, assign this characteristic
                for sample in samples:
                    if label in sample or sample in label:
                        clinical_data.iloc[row_idx, clinical_data.columns.get_loc(sample)] = char
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical features to a CSV file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'WT.4': [0.0], 'BRCA2.4': [1.0], 'BRCA1.4': [1.0], 'BRCA2.1': [1.0], 'WT.1': [0.0], 'BRCA1.1': [1.0], 'BRCA2.2': [1.0], 'BRCA1.2': [1.0], 'WT.2': [0.0], 'WT.3': [0.0], 'BRCA2.3': [1.0], 'BRCA1.3': [1.0]}
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE234017.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE234017/GSE234017_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE234017/GSE234017_series_matrix.txt.gz
Found the matrix table marker at line 70
Gene data shape: (11799, 142)
First 20 gene/probe identifiers:
['nan', 'RTS0020877', 'RTS0020879', 'RTS0020880', 'RTS0020881', 'RTS0020882', 'RTS0020883', 'RTS0020885', 'RTS0020886', 'RTS0020888', 'RTS0020892', 'RTS0020894', 'RTS0020895', 'RTS0020898', 'RTS0020904', 'RTS0020906', 'RTS0020907', 'RTS0020915', 'RTS0020917', 'RTS0020920']


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, these are not standard human gene symbols
# They appear to be probe IDs or custom identifiers (starting with "RTS") that would need 
# to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, it appears the gene symbols are in the 'ORF' column
# and the probe IDs are in the 'ID' column
print("\nExamining ID and ORF columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'ORF' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}, ORF={gene_annotation['ORF'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['ORF'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nORF column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'ORF']
{'ID': ['RTS0050057', 'RTS0020877', 'RTS0032443'], 'ORF': ['A1BG', 'A2M', 'A4GALT']}

Examining ID and ORF columns format (first 3 rows):
Row 0: ID=RTS0050057, ORF=A1BG
Row 1: ID=RTS0020877, ORF=A2M
Row 2: ID=RTS0032443, ORF=A4GALT

ORF column completeness: 1687399/1687399 rows (100.00%)


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene annotation and gene expression data to determine the mapping
# Based on the previous output and preview, we can see that:
# - In gene_annotation: 'ID' contains probe IDs (e.g., 'RTS0020877') and 'ORF' contains gene symbols (e.g., 'A2M')
# - In gene_data: The index has the same format as the 'ID' column from gene_annotation

# 2. Extract the two relevant columns from the gene annotation dataframe to create a mapping
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'ORF')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping data:")
print(preview_df(mapping_df, n=5))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols in the processed gene data:")
print(gene_data.index[:10].tolist())

# 4. Check for common issues in gene expression data
null_percentage = gene_data.isnull().mean().mean() * 100
print(f"Percentage of null values in gene data: {null_percentage:.2f}%")

# 5. Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (1687256, 2)
First 5 rows of mapping data:
{'ID': ['RTS0050057', 'RTS0020877', 'RTS0032443', 'RTS0032147', 'RTS0031938'], 'Gene': ['A1BG', 'A2M', 'A4GALT', 'A4GNT', 'AAAS']}


Gene expression data shape after mapping: (11792, 142)
First 10 gene symbols in the processed gene data:
['A1BG', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADACL4', 'AAGAB', 'AAK1', 'AAMDC']
Percentage of null values in gene data: 0.00%


Gene expression data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE234017.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (11792, 142)
Gene data shape after normalization: (11787, 142)


Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE234017.csv
Extracted clinical data shape: (1, 142)
Preview of clinical data (first 5 samples):
               GSM7441040  GSM7441041  GSM7441042  GSM7441043  GSM7441044
Breast_Cancer         0.0         0.0         0.0         0.0         0.0
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE234017.csv
Gene data columns (first 5): ['GSM7441040', 'GSM7441041', 'GSM7441042', 'GSM7441043', 'GSM7441044']
Clinical data columns (first 5): ['GSM7441040', 'GSM7441041', 'GSM7441042', 'GSM7441043', 'GSM7441044']
Found 142 common samples between gene and clinical data
Initial linked data shape: (142, 11788)
Preview of linked data (first 5 rows, first 5 columns):
            Breast_Cancer      A1BG       A2M    A4GALT     A4GNT
GSM7441040            0.0  5.544706  6.085275  5.407203  4.670237
GSM7441041            0.0  5.167737  6.691299  4.975092  3.167737
GSM7441042            0.0  6.4156

Linked data shape after handling missing values: (142, 11788)
For the feature 'Breast_Cancer', the least common label is '0.0' with 47 occurrences. This represents 33.10% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Breast_Cancer/GSE234017.csv
